In [1]:
import tensorflow as tf

## Basics

In [26]:
# computation graph
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

# prepare an init node
init = tf.global_variables_initializer()

# Initializing an Interactive Session 
session = tf.InteractiveSession()

# Initiaze all variables
init.run()

# computed the function result
result = f.eval()

# closing the Interactive Session
session.close()

In [27]:
# Managing Graphs
x1 = tf.Variable(1) # Automically gets added to the default graph
print(x1.graph is tf.get_default_graph()) # Gives True since x1 gets added to the default graph

# Reset the default graph
tf.reset_default_graph()
print(x1.graph is tf.get_default_graph()) # Gives False since we resetted the default graph

# Creating a new Graph and making it the default graph temorarily
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)
print(x2.graph is graph) # Gives True since x2 is added to the graph "graph"
print(x2.graph is tf.get_default_graph()) # Gives False since "graph" is not the default graph

True
True
True
False


## Linear Regression using TensorFlow on California Housing Dataset

In [28]:
import numpy as np
from sklearn.datasets import fetch_california_housing

In [40]:
housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_with_bias = np.c_[np.ones((m,1)),housing.data]

In [46]:
# Finding parameters (theta) using Normal Equation

# Constructing the graph
X = tf.constant(housing_data_with_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")
XT = tf.transpose(X)

theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)),XT),y)

# Execution Phase
with tf.Session() as session:
    theta_value = theta.eval()

In [64]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data_with_bias = scaler.fit_transform(housing_data_with_bias)

In [65]:
n_epochs = 1000
learning_rate = 0.01

In [66]:
tf.reset_default_graph()
# Finding parameters (theta) using Gradient Descent but Calculating gradients manually

# Constructing the graph
X = tf.constant(scaled_housing_data_with_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n+1,1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

# Calculating gradients manually
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

# prepare an init node. Does not initialize now itself
init = tf.global_variables_initializer()

with tf.Session() as session:
    session.run(init)
    print("Gradients calculated manually\n")
    for epoch in range(n_epochs):
        if epoch%100 == 0:
            print("Epoch", epoch, "MSE = ", mse.eval())
        session.run(training_op)
    print("\n")
    theta_value_manual = theta.eval()

Gradients calculated manually

Epoch 0 MSE =  9.976843
Epoch 100 MSE =  4.984743
Epoch 200 MSE =  4.898332
Epoch 300 MSE =  4.8723593
Epoch 400 MSE =  4.8540244
Epoch 500 MSE =  4.8406577
Epoch 600 MSE =  4.830878
Epoch 700 MSE =  4.8237166
Epoch 800 MSE =  4.8184667
Epoch 900 MSE =  4.8146




In [67]:
# Finding parameters (theta) using Gradient Descent but Using TensorFlow Autodiff to calculate gradients
tf.reset_default_graph()

# Constructing the graph
X = tf.constant(scaled_housing_data_with_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n+1,1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

# Using TensorFlow Autodiff to calculate gradients
gradients = tf.gradients(mse, [theta])[0]
training_op = tf.assign(theta, theta - learning_rate * gradients)

# prepare an init node. Does not initialize now itself
init = tf.global_variables_initializer()

with tf.Session() as session:
    session.run(init)
    print("Gradients calculated using Autodiff\n")
    for epoch in range(n_epochs):
        if epoch%100 == 0:
            print("Epoch", epoch, "MSE = ", mse.eval())
        session.run(training_op)
    print("\n")
    theta_value_autodiff = theta.eval()

Gradients calculated using Autodiff

Epoch 0 MSE =  7.6146326
Epoch 100 MSE =  4.895432
Epoch 200 MSE =  4.836106
Epoch 300 MSE =  4.825135
Epoch 400 MSE =  4.8191185
Epoch 500 MSE =  4.814882
Epoch 600 MSE =  4.811797
Epoch 700 MSE =  4.8095512
Epoch 800 MSE =  4.807901
Epoch 900 MSE =  4.806699




In [68]:
# Finding parameters (theta) using TensorFlow's Gradient Descent Optimizer
tf.reset_default_graph()

# Constructing the graph
X = tf.constant(scaled_housing_data_with_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n+1,1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

# Using TensorFlow's Optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

# prepare an init node. Does not initialize now itself
init = tf.global_variables_initializer()

with tf.Session() as session:
    session.run(init)
    print("Using Gradient Descent Optimizer\n")
    for epoch in range(n_epochs):
        if epoch%100 == 0:
            print("Epoch", epoch, "MSE = ", mse.eval())
        session.run(training_op)
    print("\n")
    theta_value_manual = theta.eval()

Using Gradient Descent Optimizer

Epoch 0 MSE =  9.143634
Epoch 100 MSE =  5.0988865
Epoch 200 MSE =  4.9747653
Epoch 300 MSE =  4.9286466
Epoch 400 MSE =  4.896645
Epoch 500 MSE =  4.873162
Epoch 600 MSE =  4.855802
Epoch 700 MSE =  4.842926
Epoch 800 MSE =  4.8333507
Epoch 900 MSE =  4.8261914




In [87]:
# Finding parameters (theta) using TensorFlow's Gradient Descent Optimizer and batches of data
tf.reset_default_graph()

# Constructing the graph
X = tf.placeholder(tf.float32, shape=(None, n+1),name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n+1,1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

batch_size = 100
n_batches = int(np.ceil(m / batch_size))
# Using TensorFlow's Optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

# prepare an init node. Does not initialize now itself
init = tf.global_variables_initializer()

def fetch_batch(batch_index, batch_size):
    #print(scaled_housing_data_with_bias[0:2])
    X_batch = scaled_housing_data_with_bias[batch_index : batch_index+batch_size]
    y_batch = housing.target.reshape(-1,1)[batch_index : batch_index+batch_size]
    return X_batch, y_batch

with tf.Session() as session:
    session.run(init)
    print("Using Mini-Gradient Descent Optimizer\n")
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(batch_index, batch_size)
            session.run(training_op, feed_dict={X:X_batch, y:y_batch})
        if epoch%100 == 0:
            print("Epoch", epoch, "MSE of last batch = ", mse.eval(feed_dict={X:X_batch, y:y_batch}))
    theta_value_batch = theta.eval()

Using Mini-Gradient Descent Optimizer

Epoch 0 MSE of last batch =  0.13236797
Epoch 100 MSE of last batch =  0.098801464
Epoch 200 MSE of last batch =  0.10039978
Epoch 300 MSE of last batch =  0.10022991
Epoch 400 MSE of last batch =  0.099965766
Epoch 500 MSE of last batch =  0.099701
Epoch 600 MSE of last batch =  0.099440955
Epoch 700 MSE of last batch =  0.099185936
Epoch 800 MSE of last batch =  0.098935366
Epoch 900 MSE of last batch =  0.098689385
